<a href="https://colab.research.google.com/github/PolyAgent/p1-experiments/blob/main/paper_recommender_simple_example_with_mixtral_explanations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq paperscraper sentence-transformers openai fireworks-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 29.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import userdata
FIREWORKS_API_KEY = userdata.get('FIREWORKS_API_KEY')

In [ ]:
import paperscraper

from sentence_transformers import SentenceTransformer, util
from typing import List
import pandas as pd
import numpy as np
from pprint import pprint

In [ ]:
# here we use msmarco model, which is pretrained on short query to document matching, making it a good candidate for semantic search
model = SentenceTransformer("msmarco-distilbert-dot-v5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Download a few hundred example papers

In [ ]:

MAX_NUMBER_PAPERS_PER_CAT = 200
categories = ["cat:cs.AI", "cat:cs.LG", "cat:cs.CL"]
papers_df = []
for cat in categories:
    papers_df.append(paperscraper.arxiv.get_arxiv_papers(
        cat, max_results=MAX_NUMBER_PAPERS_PER_CAT,
        search_options={'sort_by': paperscraper.arxiv.arxiv.SortCriterion.SubmittedDate}))
papers_df = pd.concat(papers_df).drop_duplicates()

Processing cat:cs.AI: 200it [00:04, 40.51it/s]
Processing cat:cs.LG: 200it [00:04, 40.64it/s]
Processing cat:cs.CL: 200it [00:04, 42.69it/s]


In [ ]:
# optionally save it
# papers_df.to_csv("2024_02_27_example.csv", index=False)

In [ ]:
papers_df['title_abs'] = papers_df['title'].str.cat(papers_df['abstract'], sep = model.tokenizer.sep_token)
papers_df.head()

,doi,date,title,authors,abstract,journal,title_abs
0,10.48550/arXiv.2402.19475,2024-02-29,The Counterfeit Conundrum: Can Code Language M...,"Alex Gu, Wen-Ding Li, Naman Jain, Theo X. Olau...",While language models are increasingly more pr...,,The Counterfeit Conundrum: Can Code Language M...
1,10.48550/arXiv.2402.19471,2024-02-29,Loose LIPS Sink Ships: Asking Questions in Bat...,"Gabriel Grand, Valerio Pepe, Jacob Andreas, Jo...",Questions combine our mastery of language with...,,Loose LIPS Sink Ships: Asking Questions in Bat...
2,10.48550/arXiv.2402.19467,2024-02-29,TV-TREES: Multimodal Entailment Trees for Neur...,"Kate Sanders, Nathaniel Weir, Benjamin Van Durme",It is challenging to perform question-answerin...,,TV-TREES: Multimodal Entailment Trees for Neur...
3,10.48550/arXiv.2402.19465,2024-02-29,Towards Tracing Trustworthiness Dynamics: Revi...,"Chen Qian, Jie Zhang, Wei Yao, Dongrui Liu, Zh...",Ensuring the trustworthiness of large language...,,Towards Tracing Trustworthiness Dynamics: Revi...
4,10.48550/arXiv.2402.19464,2024-02-29,Curiosity-driven Red-teaming for Large Languag...,"Zhang-Wei Hong, Idan Shenfeld, Tsun-Hsuan Wang...",Large language models (LLMs) hold great potent...,,Curiosity-driven Red-teaming for Large Languag...


In [ ]:
topic_of_interest = "LLM recommender systems"
query_embedding = model.encode(topic_of_interest)
corpus_embedding = model.encode(papers_df.title_abs.values)


In [ ]:
top5 = util.semantic_search(query_embedding, corpus_embedding, top_k=5)

In [ ]:
top5

[[{'corpus_id': 372, 'score': 0.8005872964859009},
  {'corpus_id': 150, 'score': 0.7999333739280701},
  {'corpus_id': 148, 'score': 0.7806745767593384},
  {'corpus_id': 83, 'score': 0.7741428017616272},
  {'corpus_id': 81, 'score': 0.7732638120651245}]]

### Print out info on the top 5 most similar papers

In [ ]:
import fireworks.client
fireworks.client.api_key = FIREWORKS_API_KEY


In [ ]:
def generate_explaination(topic_of_interest, title_abs_text):
    content = f"""
        {title_abs_text}

        Explain why the above is relevant to the topic of '{topic_of_interest}'
    """
    completion = fireworks.client.ChatCompletion.create(
        model="accounts/fireworks/models/mixtral-8x7b-instruct",
        messages=[
            {
            "role": "user",
            "content": content}
        ],
        stop=["<|im_start|>","<|im_end|>","<|endoftext|>"],
        stream=True,
        n=1,
        top_p=1,
        top_k=40,
        presence_penalty=0,
        frequency_penalty=0,
        prompt_truncate_len=1024,
        context_length_exceeded_behavior="truncate",
        temperature=0.9,
        max_tokens=4000
    )
    output = list(completion)
    msg = ""
    for chunk in output:
        curr = chunk.choices[0].delta.content
        if curr:
            msg += curr
    return msg

In [ ]:
most_similar = []
for entry in top5[0]:
    idx = entry['corpus_id']
    content=papers_df.iloc[idx].title_abs.replace('[SEP]', '\n\n')

    explanation = generate_explaination(topic_of_interest, content)
    most_similar.append(dict(
        index=idx,
        content=content,
        explanation=explanation,
        similarity_score=entry[ 'score']
    ))



In [ ]:
for entry in most_similar:
    print(entry['content'])
    print(f"similarity score: {entry[ 'similarity_score']}\n\n")
    print(""    )
    print("mixtral explanation: ")
    print(entry['explanation'].replace('.', '.\n'))
    print("\n\n")

Prospect Personalized Recommendation on Large Language Model-based Agent Platform

The new kind of Agent-oriented information system, exemplified by GPTs, urges
us to inspect the information system infrastructure to support Agent-level
information processing and to adapt to the characteristics of Large Language
Model (LLM)-based Agents, such as interactivity. In this work, we envisage the
prospect of the recommender system on LLM-based Agent platforms and introduce a
novel recommendation paradigm called Rec4Agentverse, comprised of Agent Items
and Agent Recommender. Rec4Agentverse emphasizes the collaboration between
Agent Items and Agent Recommender, thereby promoting personalized information
services and enhancing the exchange of information beyond the traditional
user-recommender feedback loop. Additionally, we prospect the evolution of
Rec4Agentverse and conceptualize it into three stages based on the enhancement
of the interaction and information exchange among Agent Items, Agent
